# Singleton Pattern Tutorial 🔐

## Table of Contents
1. [What is the Singleton Pattern?](#what-is-the-singleton-pattern)
2. [Why Do We Need It?](#why-do-we-need-it)
3. [Simple Implementation](#simple-implementation)
4. [Understanding the Implementation](#understanding-the-implementation)
5. [Thread Safety (Advanced)](#thread-safety-advanced)
6. [When to Use (and When NOT to Use)](#when-to-use-and-when-not-to-use)

## What is the Singleton Pattern?

The **Singleton Pattern** ensures that a class has **only one instance** and provides a **global way to access** that instance.

### Real-world Analogy
Think of your computer's **operating system**. No matter how many programs you run, they all interact with the **same** operating system. You can't have two Windows 11 systems running simultaneously on the same computer - there's only one, and all applications access it.

### Key Points
- ✅ **Only one instance** of the class can exist
- ✅ **Global access** to that instance
- ✅ **Lazy creation** - created only when first needed

## Why Do We Need It?

Let's see a problem that the Singleton pattern solves:

In [ ]:
# 🚫 PROBLEM: Without Singleton Pattern

class DatabaseConnection:
    """A regular class - NOT a singleton"""
    
    def __init__(self):
        print("🔌 Creating new database connection...")
        self.host = "localhost"
        self.is_connected = True
        # Imagine this is expensive (network setup, authentication, etc.)
    
    def query(self, sql: str) -> str:
        return f"Executing: {sql}"

# Let's see what happens when we create multiple instances
print("Creating database connections:")
db1 = DatabaseConnection()  # Creates connection
db2 = DatabaseConnection()  # Creates ANOTHER connection
db3 = DatabaseConnection()  # Creates YET ANOTHER connection

print(f"\nAre they the same object? {db1 is db2}")  # False!
print(f"DB1 id: {id(db1)}")
print(f"DB2 id: {id(db2)}")
print(f"DB3 id: {id(db3)}")

print("\n❌ Problems:")
print("1. We created 3 expensive database connections")
print("2. Wasting memory and resources")
print("3. Different parts of our app use different connections")
print("4. Hard to coordinate between different parts")

## Simple Implementation

Now let's fix this problem with the Singleton pattern:

In [ ]:
# ✅ SOLUTION: With Singleton Pattern

class DatabaseConnection:
    """A singleton database connection"""
    
    _instance = None  # Class variable to store the single instance
    
    def __new__(cls):
        # This method controls how new instances are created
        if cls._instance is None:
            print("🔌 Creating the ONE and ONLY database connection...")
            cls._instance = super().__new__(cls)
        else:
            print("♻️  Reusing existing database connection")
        return cls._instance
    
    def __init__(self):
        # Only initialize once
        if not hasattr(self, 'initialized'):
            self.host = "localhost"
            self.is_connected = True
            self.query_count = 0
            self.initialized = True
    
    def query(self, sql: str) -> str:
        self.query_count += 1
        return f"Query #{self.query_count}: {sql}"

# Let's test our singleton
print("Creating database connections with Singleton:")
db1 = DatabaseConnection()  # Creates the instance
db2 = DatabaseConnection()  # Returns the SAME instance
db3 = DatabaseConnection()  # Returns the SAME instance

print(f"\nAre they the same object? {db1 is db2 is db3}")  # True!
print(f"All have same id: {id(db1)} = {id(db2)} = {id(db3)}")

# Test that they share state
print("\nTesting shared state:")
result1 = db1.query("SELECT * FROM users")
result2 = db2.query("SELECT * FROM products")
result3 = db3.query("SELECT * FROM orders")

print(result1)
print(result2)
print(result3)

print(f"\nTotal queries from db1: {db1.query_count}")
print(f"Total queries from db2: {db2.query_count}")
print(f"Total queries from db3: {db3.query_count}")
print("👆 All show the same count because they're the same object!")

## Understanding the Implementation

Let's break down how our singleton works:

### What is `__new__`?

`__new__` is a special method in Python that controls **how objects are created**. It's called **before** `__init__`.

Here's the normal object creation process:

In [ ]:
# Understanding __new__ vs __init__

class RegularClass:
    def __new__(cls, name):
        print(f"1. __new__ called - creating object for {name}")
        instance = super().__new__(cls)  # Create the actual object
        return instance
    
    def __init__(self, name):
        print(f"2. __init__ called - initializing object")
        self.name = name
        print(f"3. Object ready: {self.name}")

print("Creating a regular object:")
obj = RegularClass("Alice")
print(f"Final object: {obj} with name '{obj.name}'\n")

# In our singleton, we modify __new__ to return the same instance
print("In Singleton pattern:")
print("- __new__ checks if instance already exists")
print("- If yes: return existing instance")
print("- If no: create new instance and store it")

### Step-by-Step Singleton Breakdown

In [ ]:
# Let's create a singleton with detailed logging to see what happens

class ConfigManager:
    """Singleton for managing application configuration"""
    
    _instance = None  # Class variable to store the one instance
    
    def __new__(cls):
        print(f"📞 __new__ called for {cls.__name__}")
        
        if cls._instance is None:
            print("   ✨ No instance exists yet, creating new one")
            cls._instance = super().__new__(cls)
            print(f"   📦 Created instance with id: {id(cls._instance)}")
        else:
            print(f"   ♻️  Instance already exists with id: {id(cls._instance)}")
        
        return cls._instance
    
    def __init__(self):
        print(f"🔧 __init__ called for instance {id(self)}")
        
        # Only initialize once
        if not hasattr(self, '_initialized'):
            print("   🏗️  First time initialization")
            self.settings = {"theme": "dark", "language": "en"}
            self._initialized = True
        else:
            print("   ⏭️  Already initialized, skipping")
    
    def get_setting(self, key: str) -> str:
        return self.settings.get(key, "Not found")
    
    def set_setting(self, key: str, value: str) -> None:
        self.settings[key] = value

# Now let's see exactly what happens
print("=== Creating first instance ===")
config1 = ConfigManager()

print("\n=== Creating second instance ===")
config2 = ConfigManager()

print("\n=== Creating third instance ===")
config3 = ConfigManager()

print("\n=== Testing shared state ===")
print(f"config1 theme: {config1.get_setting('theme')}")

config2.set_setting('theme', 'light')
print(f"After config2 changed theme to 'light':")
print(f"config1 theme: {config1.get_setting('theme')}")
print(f"config3 theme: {config3.get_setting('theme')}")
print("👆 All show 'light' because they're the same object!")

## Thread Safety (Advanced)

⚠️ **Important**: Our simple singleton has a problem in multi-threaded applications. Let's see why and how to fix it:

In [ ]:
# Problem: Our simple singleton is NOT thread-safe
import threading
import time
from typing import List

class UnsafeSingleton:
    """This singleton is NOT thread-safe"""
    _instance = None
    
    def __new__(cls):
        if cls._instance is None:
            # Simulate some delay (like database connection)
            time.sleep(0.1)
            cls._instance = super().__new__(cls)
        return cls._instance

# Let's test with multiple threads
instances: List[UnsafeSingleton] = []

def create_instance():
    """Function that creates an instance"""
    instance = UnsafeSingleton()
    instances.append(instance)

# Create multiple threads
threads = []
for i in range(5):
    thread = threading.Thread(target=create_instance)
    threads.append(thread)

# Start all threads at the same time
for thread in threads:
    thread.start()

# Wait for all threads to complete
for thread in threads:
    thread.join()

# Check if we have multiple instances (BAD!)
unique_ids = set(id(instance) for instance in instances)
print(f"Number of unique instances created: {len(unique_ids)}")
print(f"Instance IDs: {list(unique_ids)}")

if len(unique_ids) > 1:
    print("❌ PROBLEM: Multiple instances were created!")
else:
    print("✅ GOOD: Only one instance was created")

### Thread-Safe Singleton Solution

In [ ]:
# ✅ SOLUTION: Thread-safe singleton

class ThreadSafeSingleton:
    """A thread-safe singleton implementation"""
    
    _instance = None
    _lock = threading.Lock()  # This prevents multiple threads from creating instances
    
    def __new__(cls):
        # First check without lock (for performance)
        if cls._instance is None:
            # Now acquire lock before creating
            with cls._lock:
                # Double-check after acquiring lock
                if cls._instance is None:
                    print(f"🔐 Creating thread-safe instance in thread {threading.current_thread().name}")
                    time.sleep(0.1)  # Simulate expensive operation
                    cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not hasattr(self, '_initialized'):
            self.value = f"Created by {threading.current_thread().name}"
            self._initialized = True

# Test thread-safe version
safe_instances: List[ThreadSafeSingleton] = []

def create_safe_instance():
    """Function that creates a thread-safe instance"""
    instance = ThreadSafeSingleton()
    safe_instances.append(instance)

# Create multiple threads
safe_threads = []
for i in range(5):
    thread = threading.Thread(target=create_safe_instance, name=f"Thread-{i}")
    safe_threads.append(thread)

# Start all threads
for thread in safe_threads:
    thread.start()

# Wait for completion
for thread in safe_threads:
    thread.join()

# Check results
unique_safe_ids = set(id(instance) for instance in safe_instances)
print(f"\nThread-safe test results:")
print(f"Number of unique instances: {len(unique_safe_ids)}")
print(f"All instances have same value: {safe_instances[0].value}")

if len(unique_safe_ids) == 1:
    print("✅ SUCCESS: Thread-safe singleton works correctly!")
else:
    print("❌ FAILED: Still creating multiple instances")

### Understanding Thread Safety

**What's a Lock?**
- A `threading.Lock()` is like a **bathroom door lock**
- Only **one thread** can "enter" (execute the code) at a time
- Other threads must **wait** until the first thread "exits" (releases the lock)

**Double-Check Locking:**
1. **First check**: Quick check without lock (for performance)
2. **Acquire lock**: If needed, get exclusive access
3. **Second check**: Check again (another thread might have created it while we waited)
4. **Create**: Only create if still needed

## When to Use (and When NOT to Use)

### ✅ Good Use Cases:

1. **Database Connections** - Expensive to create, should be shared
2. **Configuration Settings** - One source of truth for app settings
3. **Logging** - All parts of app should log to same system
4. **Caching** - One cache shared across the application

### ❌ Bad Use Cases (Avoid Singleton When):

1. **You just want global variables** - Use modules instead
2. **Testing is important** - Singletons are hard to test
3. **You might need multiple instances later** - Don't paint yourself into a corner
4. **Simple objects** - Don't over-engineer

### Example: When NOT to use Singleton

In [ ]:
# ❌ BAD: Using singleton for simple data
class UserSingleton:
    """DON'T do this - user objects should not be singletons!"""
    _instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

# This is broken! What if we want multiple users?
user1 = UserSingleton("Alice", "alice@example.com")
user2 = UserSingleton("Bob", "bob@example.com")  # This overwrites Alice!

print(f"User1 name: {user1.name}")  # Will be "Bob"!
print(f"User2 name: {user2.name}")  # Also "Bob"
print(f"Same object: {user1 is user2}")  # True - this is bad!

print("\n✅ BETTER: Use regular classes for data objects")

class User:
    """Regular class - much better for data objects"""
    def __init__(self, name: str, email: str):
        self.name = name
        self.email = email

user_a = User("Alice", "alice@example.com")
user_b = User("Bob", "bob@example.com")

print(f"UserA name: {user_a.name}")  # Alice
print(f"UserB name: {user_b.name}")  # Bob
print(f"Different objects: {user_a is not user_b}")  # True - this is good!

In [ ]:
# Solution - don't peek until you've tried!
import threading
from typing import List, Dict
from datetime import datetime

class Logger:
    """Thread-safe Logger singleton"""
    
    _instance = None
    _lock = threading.Lock()
    
    def __new__(cls):
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = super().__new__(cls)
        return cls._instance
    
    def __init__(self):
        if not hasattr(self, '_initialized'):
            self._logs: List[Dict[str, str]] = []
            self._log_lock = threading.Lock()  # Separate lock for logging
            self._initialized = True
    
    def _log(self, level: str, message: str):
        """Internal method to add log entry"""
        with self._log_lock:
            timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            log_entry = {
                "timestamp": timestamp,
                "level": level,
                "message": message
            }
            self._logs.append(log_entry)
    
    def info(self, message: str):
        """Log an info message"""
        self._log("INFO", message)
    
    def warning(self, message: str):
        """Log a warning message"""
        self._log("WARNING", message)
    
    def error(self, message: str):
        """Log an error message"""
        self._log("ERROR", message)
    
    def get_logs(self) -> List[Dict[str, str]]:
        """Get all log messages"""
        with self._log_lock:
            return self._logs.copy()  # Return a copy to prevent external modification

# Test the solution
logger1 = Logger()
logger2 = Logger()

logger1.info("Application started")
logger2.warning("This is a warning")
logger1.error("Something went wrong")

print(f"Are they the same? {logger1 is logger2}")
print(f"Number of logs: {len(logger1.get_logs())}")
print("\nAll logs:")
for log in logger1.get_logs():
    print(f"[{log['timestamp']}] {log['level']}: {log['message']}")

print("\n✅ Logger singleton working correctly!")